# This is an auxiliar model to load the info to Pinecone

In [ ]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import pickle
import os
from streamlit_extras.add_vertical_space import add_vertical_space
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain_pinecone import Pinecone
index_name = 'thesis-model-1'

In [ ]:
pdf_reader = PdfReader("user manual.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

# Split text in smaller chunks (checked)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  # Keep context
    length_function=len,
)
chunks = text_splitter.split_text(text=text)
# chunks = text_splitter.create_documents([text])

In [10]:
# Computing Embeddings (with Hughing Face)
emb_model = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceHubEmbeddings(repo_id=emb_model,
                                        huggingfacehub_api_token=os.getenv('HUGGINGFACEHUB_API_TOKEN'))

# Store the embeddings

vectorStore = Pinecone.from_texts(
    chunks, embeddings, index_name=index_name)

c:\Anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
You're using a different task than the one specified in the repository. Be sure to know what you're doing :)
2024-03-12 06:18:44.291 WARNING huggingface_hub.inference_api: You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [15]:
query = "Cómo ponerle las pilas al wii?"
docs = vectorStore.similarity_search(query=query, k=3)
docs
        # st.write(docs)  # write nearest chunks
        # llm_model = OpenAI(temperature=0.9, model_name='gpt-3.5-turbo')
        # chain = load_qa_chain(llm=llm_model, chain_type="stuff")
        # with get_openai_callback() as cb:
        #     response = chain.run(input_documents=docs, question=query)
        #     # TODO: Modify prompts, and review current prompts

        #     # To know costs
        #     print(cb)
        # st.write(response)

[Document(page_content='Cómo instalar las pilas del Wii Remote   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 143\nColocación de la correa del Wii Remote   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 144\nCómo usar el Nunchuk   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 145\nCómo desconectar el Nunchuk  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 146\n \nConfiguración de la consola\n \nConfiguración de la consola . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 148\nCómo cambiar la configuración . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 148\nAjustes de usuario   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 149\nCómo registrar usuarios   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [18]:
index_name = "thesis-model-1"
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [19]:
query = "Cómo ponerle las pilas al wii?"
docs = vectorStore.similarity_search(query=query, k=3)
docs

[Document(page_content='Cómo instalar las pilas del Wii Remote   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 143\nColocación de la correa del Wii Remote   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 144\nCómo usar el Nunchuk   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 145\nCómo desconectar el Nunchuk  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 146\n \nConfiguración de la consola\n \nConfiguración de la consola . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 148\nCómo cambiar la configuración . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 148\nAjustes de usuario   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 149\nCómo registrar usuarios   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .